In [2]:
import os
import time
import multiprocessing as mp
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
import func4behav as fb
import itertools
import ast
import imp
imp.reload(fb)

C:\Users\Tianyi Zheng\AppData\Local\Temp\ipykernel_26348\3756806229.py:12: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp


<module 'func4behav' from 'c:\\Users\\Tianyi Zheng\\OneDrive - neuron.t.u-tokyo.ac.jp\\Documents\\zheng\\mywork\\attention_tES\\tes-attention\\processing\\behavior\\func4behav.py'>

In [2]:
def auto_sig(size, watch_cases, watch_idxs):
    record = pd.DataFrame(columns=['try id', 'Real picks', 'Sham picks', 'p values', 'Judge'])
    all_real = [1, 3, 4, 5, 9, 12, 13, 17, 18]
    all_sham = [2, 6, 7, 8, 10, 11, 14, 15, 16]
    real_comb = [list(comb) for comb in itertools.combinations(all_real, size)]
    sham_comb = [list(comb) for comb in itertools.combinations(all_sham, size)]

    start_time = time.time()
    try_id = 0

    for real_to_pick in real_comb:
        for sham_to_pick in sham_comb:
            p_values = fb.auto_compare(real_to_pick, sham_to_pick, watch_cases, watch_idxs)
            record.at[try_id, 'try id'] = try_id
            record.at[try_id, 'Real picks'] = real_to_pick
            record.at[try_id, 'Sham picks'] = sham_to_pick
            record.at[try_id, 'p values'] = p_values.values.tolist()
            record.at[try_id, 'Judge'] = [1 if all(i <= 0.05 for i in sublist if i is not None) else 0 for sublist in p_values.values]

            judge_matrix = p_values.applymap(lambda x: 1 if x <= 0.05 else 0)
            record.at[try_id, 'Judge'] = judge_matrix.values.tolist()

            try_id += 1

    end_time = time.time()
    elapsed_time = end_time - start_time  # Calculate the difference
    print(f"The program took {elapsed_time} seconds to run.")
    record.to_csv('../../../docs/report/record_'+str(size)+'.csv', index=False)

    return elapsed_time

In [3]:
watch_cases = ['endo', 'exo', 'endo valid', 'exo valid', 'endo invalid', 'exo invalid', 'endo stim left', 'endo stim right', 'exo stim left', 'exo stim right']
watch_idxs = ["RT mean shorten", "RT median shorten", "RT mean shorten %", "RT median shorten %"]
size = 7
auto_sig(size, watch_cases, watch_idxs)

The program took 6706.270402908325 seconds to run.


6706.270402908325

### View result

In [3]:
record_path = os.path.join('..', '..', '..', 'docs', 'report', 'record_7'+'.csv')
record = pd.read_csv(record_path)

# Convert the 'Judge' column from string representation to actual list of lists
record['Judge'] = record['Judge'].apply(ast.literal_eval)

# Filter the dataframe
record = record[record['Judge'].apply(lambda row: any(1 in sublist for sublist in row))]

# Define a function to sum the sublist elements column-wise
def sum_sublist_elements(lst):
    return [sum(x) for x in zip(*lst)]

# Apply this function to the 'Judge' column and create a new column 'Judge_sum'
record['Judge_sum'] = record['Judge'].apply(sum_sublist_elements)
record


,try id,Real picks,Sham picks,p values,Judge,Judge_sum
190,190,"[1, 3, 4, 5, 9, 17, 18]","[2, 6, 7, 10, 11, 14, 15]","[[0.31759906759906764, 0.09731934731934733, 0....","[[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0,...","[0, 1, 0, 0]"
192,192,"[1, 3, 4, 5, 9, 17, 18]","[2, 6, 7, 10, 11, 15, 16]","[[0.16491841491841494, 0.05303030303030303, 0....","[[0, 0, 0, 1], [0, 0, 0, 0], [0, 0, 0, 0], [0,...","[0, 0, 0, 1]"
622,622,"[1, 3, 5, 9, 12, 17, 18]","[2, 6, 7, 10, 11, 14, 15]","[[0.31759906759906764, 0.09731934731934733, 0....","[[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0,...","[0, 1, 0, 0]"
624,624,"[1, 3, 5, 9, 12, 17, 18]","[2, 6, 7, 10, 11, 15, 16]","[[0.16491841491841494, 0.05303030303030303, 0....","[[0, 0, 0, 1], [0, 0, 0, 0], [0, 0, 0, 0], [0,...","[0, 0, 0, 1]"
658,658,"[1, 3, 5, 9, 13, 17, 18]","[2, 6, 7, 10, 11, 14, 15]","[[0.31759906759906764, 0.09731934731934733, 0....","[[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0,...","[0, 1, 0, 0]"
660,660,"[1, 3, 5, 9, 13, 17, 18]","[2, 6, 7, 10, 11, 15, 16]","[[0.16491841491841494, 0.05303030303030303, 0....","[[0, 0, 0, 1], [0, 0, 0, 0], [0, 0, 0, 0], [0,...","[0, 0, 0, 1]"
1192,1192,"[3, 4, 9, 12, 13, 17, 18]","[2, 6, 7, 8, 10, 14, 16]","[[0.31759906759906764, 0.6200466200466201, 0.3...","[[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 1, 0], [0,...","[0, 0, 1, 0]"
1210,1210,"[3, 4, 9, 12, 13, 17, 18]","[2, 7, 8, 10, 11, 14, 16]","[[0.31759906759906764, 0.6200466200466201, 0.3...","[[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 1, 0], [0,...","[0, 0, 1, 0]"
1212,1212,"[3, 4, 9, 12, 13, 17, 18]","[2, 7, 8, 10, 14, 15, 16]","[[0.31759906759906764, 0.6200466200466201, 0.3...","[[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 1, 0], [0,...","[0, 0, 1, 0]"


In [5]:
# record_path = os.path.join('..', '..', '..', 'docs', 'report', 'record_8.csv')
# record = pd.read_csv(record_path)

# # Use apply with ast.literal_eval to convert string representation of list to actual list
# record['Judge'] = record['Judge'].apply(ast.literal_eval)
# record['p value'] = record['p value'].apply(ast.literal_eval)

# record['p1'] = record['p value'].apply(lambda x: x[0] if isinstance(x, list) and len(x) > 0 else None)
# record['p2'] = record['p value'].apply(lambda x: x[1] if isinstance(x, list) and len(x) > 1 else None)
# record['Judge 1'] = record['p1'].apply(lambda x: 1 if sum(i <= 0.05 for i in x) == 3 else 0)
# record['Judge 2'] = record['p2'].apply(lambda x: 1 if sum(i <= 0.05 for i in x) == 4 else 0)

# # # Filter rows where 'Judge' column is [1, 1]
# # record = record[record['Judge'].apply(lambda x: x == [0, 1])]
# record = record[record['Judge 1'] == 1]
# # record = record[record['Judge 2'] == 1]

# # Display the updated DataFrame
# display(record)